In [1]:
# modified to add locations and prep for database

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../../data/raw/ZHVI/ZHVI_April24.csv', dtype=str)

## Transpose date columns

Assumption is that there is an entry for every month and every zipcode... this must be true since the date columns all exist (empty values are nan)

In [3]:
# predetermine size of array
# 3 arrays of length: (# of timeseries entries (date columms)) * # unique zips
arr = np.empty([3,(len(df.columns) - 9) * df['RegionName'].nunique()], dtype=object)

In [4]:
a = 0
b = (len(df.columns) - 9)

# for every zip, transpose date columns and repeat zip code in 3rd column
for ind, row in df.iterrows():
  timeseries = row.iloc[9:].T

  arr[0,a:b] = timeseries.index.values
  arr[1,a:b] = timeseries.values
  arr[2,a:b] = np.repeat(row['RegionName'], len(timeseries.index.values))
  a += len(timeseries)# length of time in dataset
  b += len(timeseries)

In [5]:
temp_df = pd.DataFrame(arr.T)
temp_df.columns = ['Date', 'ZHVI', 'ZipCode']

In [6]:
temp_df

,Date,ZHVI,ZipCode
0,2000-01-31,213463.22176577282,77494
1,2000-02-29,213648.48856106598,77494
2,2000-03-31,214143.5435946286,77494
3,2000-04-30,215143.44785558843,77494
4,2000-05-31,215179.21426169714,77494
...,...,...,...
7693611,2023-12-31,196271.14791287834,50160
7693612,2024-01-31,197149.51056180146,50160
7693613,2024-02-29,197990.45808887473,50160
7693614,2024-03-31,199040.41287595208,50160


In [7]:
df.iloc[:,1:9]

,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName
0,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County
1,2,08701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County
2,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County
3,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County
4,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County
...,...,...,...,...,...,...,...,...
26343,39992,52163,zip,IA,IA,Protivin,NaN,Howard County
26344,39992,46799,zip,IN,IN,Zanesville,"Bluffton, IN",Wells County
26345,39992,22731,zip,VA,VA,Aroda,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Madison County
26346,39992,26576,zip,WV,WV,Farmington,"Fairmont, WV",Marion County


In [8]:
df = temp_df.merge(df.iloc[:,1:9], how='left', left_on='ZipCode', right_on='RegionName')

In [9]:
df

,Date,ZHVI,ZipCode,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName
0,2000-01-31,213463.22176577282,77494,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County
1,2000-02-29,213648.48856106598,77494,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County
2,2000-03-31,214143.5435946286,77494,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County
3,2000-04-30,215143.44785558843,77494,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County
4,2000-05-31,215179.21426169714,77494,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County
...,...,...,...,...,...,...,...,...,...,...,...
7693611,2023-12-31,196271.14791287834,50160,39992,50160,zip,IA,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County
7693612,2024-01-31,197149.51056180146,50160,39992,50160,zip,IA,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County
7693613,2024-02-29,197990.45808887473,50160,39992,50160,zip,IA,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County
7693614,2024-03-31,199040.41287595208,50160,39992,50160,zip,IA,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County


In [10]:
df = df.drop(['RegionName', 'RegionType', 'StateName'], axis=1)

In [11]:
df.to_csv('../../data/formatted/ZHVI/ZHVI_April24_Formatted.csv', index= False)

## Align location name with uszipcode directory

Location names given by Zillow don't always match location names based off the zip code when searched through uszipcode. Match location names so that it is accurate when being searched on the realestatetimeseries.com site.

In [12]:
%pip install uszipcode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: C:\Users\Steve\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
from uszipcode import SearchEngine

sr = SearchEngine()

C:\Users\Steve\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [14]:
zips = df['ZipCode'].unique()
zips = pd.DataFrame(zips, columns=['Zip'])
zips

,Zip
0,77494
1,08701
2,77449
3,11368
4,77084
...,...
26343,52163
26344,46799
26345,22731
26346,26576


In [15]:
zips['Location'] = zips.apply(lambda x: sr.by_zipcode(x['Zip']).major_city + ', ' + sr.by_zipcode(x['Zip']).state + " " + str(x['Zip']), axis=1)

In [16]:
df = df.merge(zips, how='inner', left_on='ZipCode', right_on='Zip')

In [17]:
df = df.drop(['Zip'], axis=1)
df

,Date,ZHVI,ZipCode,SizeRank,State,City,Metro,CountyName,Location
0,2000-01-31,213463.22176577282,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
1,2000-02-29,213648.48856106598,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
2,2000-03-31,214143.5435946286,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
3,2000-04-30,215143.44785558843,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
4,2000-05-31,215179.21426169714,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
...,...,...,...,...,...,...,...,...,...
7693611,2023-12-31,196271.14791287834,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"
7693612,2024-01-31,197149.51056180146,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"
7693613,2024-02-29,197990.45808887473,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"
7693614,2024-03-31,199040.41287595208,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"


In [18]:
df['ZHVI'] = df['ZHVI'].astype(float)
df['Date'] = pd.to_datetime(df['Date'])

In [19]:
df['ZHVI'] = df['ZHVI'].round()

In [20]:
df

,Date,ZHVI,ZipCode,SizeRank,State,City,Metro,CountyName,Location
0,2000-01-31,213463.0,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
1,2000-02-29,213648.0,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
2,2000-03-31,214144.0,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
3,2000-04-30,215143.0,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
4,2000-05-31,215179.0,77494,1,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,"Katy, TX 77494"
...,...,...,...,...,...,...,...,...,...
7693611,2023-12-31,196271.0,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"
7693612,2024-01-31,197150.0,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"
7693613,2024-02-29,197990.0,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"
7693614,2024-03-31,199040.0,50160,39992,IA,Martensdale,"Des Moines-West Des Moines, IA",Warren County,"Martensdale, IA 50160"


In [21]:
ZHVI_Table_May23 = df[['Date', 'ZHVI', 'ZipCode']]

In [22]:
Locations_Table_May23 = df.iloc[:,2:].drop_duplicates()

In [23]:
Locations_Json_May23 = df[['ZipCode', 'Location']].drop_duplicates()

In [25]:
ZHVI_Table_May23.to_csv('../../data/site/ZHVI_Table_April24.csv', index=False)

In [26]:
Locations_Table_May23.to_csv('../../data/site/Locations_Table_April24.tsv', sep='\t', index=False)

In [27]:
Locations_Json_May23.to_json('locations_April24.json', orient='records')